In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime as dt
from selenium.common.exceptions import StaleElementReferenceException, WebDriverException
import re
from bs4 import BeautifulSoup
from time import sleep

In [2]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

In [3]:
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title

In [4]:
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [5]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
try:
    dates_objects[prev_day-1].click()
except:
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    dates_objects[prev_day-1].click()

In [6]:
def get_data_event(event):
    cleaned_event = re.sub('<tbody>|<tr>|<td>|</tbody>|</tr>|</td>|<td colspan="2">|\t','', event)
    nsymbols_data = 16
    date = cleaned_event[:nsymbols_data]
    cleaned_event_ = cleaned_event[nsymbols_data:].strip()
    splitted_event = cleaned_event_.split(';')
    players = splitted_event[0]
    cleaned_event_ = splitted_event[1].strip()
    if 'Победа' in cleaned_event:
        winner = cleaned_event_.split(', ')[1].split(',')[0].strip()
        splitted_event = cleaned_event_.split(', ')[0].split('  ')
        event_ = ' '.join(splitted_event[1:3])
        score = splitted_event[-1]
    else:
        winner = ''
        event_ = ' '.join(cleaned_event_.split('  ')[1:3])
        score = ''
    return date, event_, players, winner, score


def get_data_table(tables):
    page_tables = []
    for str_table in tables:
        soup = BeautifulSoup(str_table, 'html.parser')
        cleaned_table_data = [
            re.sub('<tr> | </tr>| class="c1"| class="c2"', '', str(x)) for x in soup.find_all('tr')[1:-1]
        ]
        columns = soup.find_all('th')
        columns = [re.sub('<th> | </th>', '', str(x)) for x in columns]
        table = []
        for str_ in cleaned_table_data: 
            soup_str = BeautifulSoup(str_, 'html.parser')
            table.append([x.text for x in soup_str.find_all('td')])
        table = pd.DataFrame(data=table, columns=columns)
        page_tables.append(table)
    return page_tables


def restart_page(driver):
    tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
    tennis_btn[0].click()
    # looking for available date
    previous_date = dt.datetime.today() - dt.timedelta(days=1)
    prev_day = previous_date.day
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    try:
        dates_objects[prev_day-1].click()
    except:
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        dates_objects[prev_day-1].click()

In [7]:
bcs = ['tab_1x', 'tab_bc', 'tab_fb', 'tab_mf']
data_events = {bc: [] for bc in bcs}
data_tables = {bc: [] for bc in bcs}
for bc in bcs:
    sleep(1)
    driver.find_elements(By.ID, bc)[0].click()
    sleep(1)
    npages = driver.find_elements(By.CLASS_NAME, 'pages')
    # pages iterating
    for i in range(len(npages)):
        try:
            driver.execute_script("arguments[0].click();", npages[i])
        except (StaleElementReferenceException, WebDriverException):
            npages = driver.find_elements(By.CLASS_NAME, 'pages')
            driver.execute_script("arguments[0].click();", npages[i])    
        sleep(1)
        html_source_code = driver.execute_script("return document.body.innerHTML;")
        html_soup = BeautifulSoup(html_source_code, 'html.parser')
        tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
        # events parsing
        events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
        events = tbody_data[events_indexes]
        # data_events.append([get_data_event(x) for x in events])
        data_events[bc].append([get_data_event(x) for x in events])
        # tables parsing
        tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
        # data_tables.append(get_data_table(tables))
        data_tables[bc].append(get_data_table(tables))

In [8]:
np.vstack(data_events['tab_1x']).shape

(57, 5)

In [9]:
np.hstack(np.array(data_tables['tab_1x'], dtype='object')).size

C:\Users\dimaz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\shape_base.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


57

In [10]:
50 + 55 + 42

147

correct -> 50, 55, 42

# Calendar switcher

In [35]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
try:
    dates_objects[prev_day-1].click()
except:
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    dates_objects[prev_day-1].click()

In [80]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()


# Page switcher
current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text
first_month_flg = True
while current_page_month != 'Декабрь' or current_page_year != '2019':
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    n_range = range(len(dates_objects)-1, -1, -1)
    for i in n_range:
        if first_month_flg:
            data_availability_text = driver.find_elements(By.ID, 'response_1x')[0].text
            if 'Для доступа к движению линий' in data_availability_text:
                continue
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        try:
            dates_objects[i].click()
        except:
            dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
            dates_objects[i].click()
    left_triangle = driver.find_elements(By.LINK_TEXT, 'Назад')
    left_triangle[0].click()
    first_month_flg = False
    current_page_month = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-month')[0].text
    current_page_year = driver.find_elements(By.CLASS_NAME, 'ui-datepicker-year')[0].text

23 Май 2023
22 Май 2023
21 Май 2023
20 Май 2023


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00248893+48451]
	(No symbol) [0x001DB8A1]
	(No symbol) [0x000E5058]
	(No symbol) [0x000CD073]
	(No symbol) [0x0012DEBB]
	(No symbol) [0x0013BFD3]
	(No symbol) [0x0012A0B6]
	(No symbol) [0x00107E08]
	(No symbol) [0x00108F2D]
	GetHandleVerifier [0x004A8E3A+2540266]
	GetHandleVerifier [0x004E8959+2801161]
	GetHandleVerifier [0x004E295C+2776588]
	GetHandleVerifier [0x002D2280+612144]
	(No symbol) [0x001E4F6C]
	(No symbol) [0x001E11D8]
	(No symbol) [0x001E12BB]
	(No symbol) [0x001D4857]
	BaseThreadInitThunk [0x76A37D59+25]
	RtlInitializeExceptionChain [0x7778B74B+107]
	RtlClearBits [0x7778B6CF+191]
	(No symbol) [0x00000000]


'Движение линий в 1xstavka за 27-05-2023. Теннис\nТеннис (57 матчей на Дату: 27.05.2023 г.)\nATP. Женева\n27.05.2023 16:00 Николас Харри - Григор Димитров; (ID=528) Теннис : ATP. Женева\nИтог матча: Счет: 2:0 (7:6,6:1), Победа: Николас Харри, Тотал Матча: 20, (ИТ1=13, ИТ2=7, D=6)\nДата-время скана линии П1 П2 1Х 2Х Х 12 ТМ ТМ бол ТМ мен ФОРА 1 Коэф Ф1 ФОРА 2 Коэф Ф2 Инд ТОТ 1 ИТ1 бол ИТ1 мен Инд ТОТ 2 ИТ2 бол ИТ2 мен Прим.\n26 май 21:40 2,28 1,53 22,5 1,88 1,78 2,5 1,83 -2,5 1,83 11,5 1,59 2,15 12,5 1,8 1,86\n26 май 21:50 2,13 1,61 22,5 1,83 1,83 1,5 1,91 -1,5 1,76 12,5 2,06 1,65 12,5 1,77 1,9\n26 май 22:00 1,93 1,74 23,5 1,9 1,76 1,5 1,75 -1,5 1,92 12,5 1,77 1,9 12,5 1,7 1,98\n26 май 22:20 1,78 2,04 23,5 2 1,81 -0,5 1,78 0,5 1,88 12,5 1,73 2,04 12,5 1,83 1,91\n26 май 22:30 1,76 2,07 23,5 2 1,81 -1,5 1,9 1,5 1,77 12,5 1,71 2,06 12,5 1,83 1,91\n26 май 22:50 1,74 2,1 23,5 2 1,81 -1,5 1,87 1,5 1,79 12,5 1,7 2,08 12,5 1,84 1,9\n26 май 23:00 1,74 2,1 23,5 1,97 1,83 -1,5 1,86 1,5 1,8 12,5 1,